# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvzf spark-3.3.0-bin-hadoop3.tgz
!pip install findspark

import os
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("IPL Analysis").getOrCreate()

from IPython.display import clear_output 
clear_output()

print(spark.sparkContext.appName ,"is Running..")

IPL Analysis is Running..


# Data Exploration

In [2]:
from pyspark.sql.functions import year

In [3]:
df1=spark.read.option("inferSchema", "true").option("header", "true").csv("/content/drive/MyDrive/Colab Notebooks/SparkData/IPL Matches.csv")

In [4]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- neutral_venue: integer (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



In [5]:
df1.count()

816

In [6]:
df1.head(10)

[Row(id=335982, city='Bangalore', date=datetime.datetime(2008, 4, 18, 0, 0), player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen'),
 Row(id=335983, city='Chandigarh', date=datetime.datetime(2008, 4, 19, 0, 0), player_of_match='MEK Hussey', venue='Punjab Cricket Association Stadium, Mohali', neutral_venue=0, team1='Kings XI Punjab', team2='Chennai Super Kings', toss_winner='Chennai Super Kings', toss_decision='bat', winner='Chennai Super Kings', result='runs', result_margin='33', eliminator='N', method='NA', umpire1='MR Benson', umpire2='SL Shastri'),
 Row(id=335984, city='Delhi', date=datetime.datetime(2008, 4, 19, 0, 0), player_of_match='MF Maharoof', venue='Feroz Shah Kotla', neutral_venue

# Counting the number of matches played in each city.

In [7]:
df2=df1.groupBy('city').count()
df2=df2.orderBy("count",ascending=False)

df2.show(truncate=False)

+-------------+-----+
|city         |count|
+-------------+-----+
|Mumbai       |101  |
|Kolkata      |77   |
|Delhi        |74   |
|Bangalore    |65   |
|Hyderabad    |64   |
|Chennai      |57   |
|Chandigarh   |56   |
|Jaipur       |47   |
|Pune         |38   |
|Abu Dhabi    |29   |
|Dubai        |26   |
|Durban       |15   |
|Bengaluru    |15   |
|Visakhapatnam|13   |
|NA           |13   |
|Ahmedabad    |12   |
|Sharjah      |12   |
|Centurion    |12   |
|Rajkot       |10   |
|Dharamsala   |9    |
+-------------+-----+
only showing top 20 rows



# Counting the number of matches won by each team in 2014.

In [8]:
df3=df1.filter(year(df1.date) == "2014")
df3=df3.groupBy('winner').count()
df3=df3.orderBy("count",ascending=False)

df3.show(truncate=False)

+---------------------------+-----+
|winner                     |count|
+---------------------------+-----+
|Kings XI Punjab            |12   |
|Kolkata Knight Riders      |11   |
|Chennai Super Kings        |10   |
|Mumbai Indians             |7    |
|Rajasthan Royals           |7    |
|Sunrisers Hyderabad        |6    |
|Royal Challengers Bangalore|5    |
|Delhi Daredevils           |2    |
+---------------------------+-----+



# Finding the Player who won the most MOM award.

In [9]:
df4=df1.groupBy('player_of_match').count()
df4=df4.orderBy("count",ascending=False)

df4.show(1,truncate=False)

+---------------+-----+
|player_of_match|count|
+---------------+-----+
|AB de Villiers |23   |
+---------------+-----+
only showing top 1 row



# Find the top 10 umpires who was present in most of the matches.

In [10]:
df5=df1.select('umpire1').union(df1.select('umpire2'))
df5=df5.filter(df5.umpire1!='NA')
df5=df5.groupBy('umpire1').count()
df5=df5.orderBy("count",ascending=False)

df5.show(10,truncate=False)

+---------------+-----+
|umpire1        |count|
+---------------+-----+
|S Ravi         |121  |
|HDPK Dharmasena|94   |
|AK Chaudhary   |87   |
|C Shamshuddin  |82   |
|M Erasmus      |65   |
|CK Nandan      |57   |
|Nitin Menon    |57   |
|SJA Taufel     |55   |
|Asad Rauf      |51   |
|VA Kulkarni    |50   |
+---------------+-----+
only showing top 10 rows



# Count of teams who won the Match after winning the Toss.

In [11]:
df6=df1.filter(df1.toss_winner==df1.winner)
df6=df6.groupBy('winner').count()
df6=df6.orderBy("count",ascending=False)

df6.show(truncate=False)

+---------------------------+-----+
|winner                     |count|
+---------------------------+-----+
|Mumbai Indians             |61   |
|Chennai Super Kings        |61   |
|Kolkata Knight Riders      |55   |
|Rajasthan Royals           |44   |
|Royal Challengers Bangalore|43   |
|Kings XI Punjab            |36   |
|Delhi Daredevils           |35   |
|Sunrisers Hyderabad        |29   |
|Deccan Chargers            |19   |
|Delhi Capitals             |10   |
|Gujarat Lions              |10   |
|Rising Pune Supergiant     |5    |
|Kochi Tuskers Kerala       |4    |
|Rising Pune Supergiants    |3    |
|Pune Warriors              |3    |
+---------------------------+-----+

